In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from scipy.stats import randint
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


Loading the Movie Data from extracted TMDB

In [2]:
# Define a list of file paths for CSV files to be loaded
file_paths = [
    'Yearly\movies2013.csv',
    'Yearly\movies2014.csv',
    'Yearly\movies2015.csv',
    'Yearly\movies2016.csv',
    'Yearly\movies2017.csv',
    'Yearly\movies2018.csv',
    'Yearly\movies2019.csv',
    'Yearly\movies2020.csv',
    'Yearly\movies2021.csv',
    'Yearly\movies2022.csv',
    'Yearly\movies2023.csv'
]

# Initialize an empty list to store dataframes
dataframes = []

# Load each CSV file into a dataframe and store them in the 'dataframes' list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all dataframes into a single dataframe
TMDB_df = pd.concat(dataframes)

# Remove duplicate rows based on the 'imdb_id' column, keeping the first occurrence
TMDB_df.drop_duplicates(subset=['imdb_id'], keep='first', inplace=True)

# Now, 'TMDB_df' contains the merged data from all CSV files without duplicates


In [3]:
TMDB_df.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'keywords', 'cast'],
      dtype='object')

In [4]:
# Read the 'Online 2017 data.csv' file into a dataframe
online = pd.read_csv('Yearly\Online 2017 data.csv')

# Rename the 'Keywords' column to 'keywords' to ensure successful merging
online.rename(columns={'Keywords': 'keywords'}, inplace=True)

# Concatenate the 'online' dataframe with 'TMDB_df'
TMDB_df = pd.concat([online, TMDB_df])

# Remove duplicate rows based on the 'imdb_id' column, keeping the first occurrence
TMDB_df.drop_duplicates(subset=['imdb_id'], keep='first', inplace=True)

# Now, 'TMDB_df' contains the merged data with the online data without duplicates
TMDB_df[TMDB_df['revenue']!=0]

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,title,keywords,cast,crew,revenue,adult,backdrop_path,video,vote_average,vote_count
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,NaN,NaN,NaN,NaN,NaN
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,NaN,NaN,NaN,NaN,NaN
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,NaN,NaN,NaN,NaN,NaN
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,NaN,NaN,NaN,NaN,NaN
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28446,800227,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://a24films.com/films/all-dirt-roads-taste...,tt11534720,en,All Dirt Roads Taste of Salt,Tender caresses and enveloping embraces are po...,6.058000,...,All Dirt Roads Taste of Salt,"['woman director', 'visual poem', 'mississippi']","['Charleen McClure', 'Sheila Atim', 'Moses Ing...",NaN,41000,False,/p65KvZIFrlYxB3LZU2e5WtYvmfZ.jpg,False,0.0,0.0
29447,1187038,NaN,6,"[{'id': 9648, 'name': 'Mystery'}, {'id': 80, '...",https://www.pioneerpicture.com,tt27097230,kn,Manduka,A girl suffering from schizophrenia accidental...,3.045000,...,Manduka,"['india', 'kannada', 'accidental crime', 'sand...","['Anjala', 'Madhan', 'Bhushan', 'Mysore raju',...",NaN,10,False,/oXLzPSTw5YoAUU7XcXf88BjICq2.jpg,False,0.0,0.0
29597,1065835,NaN,5404446,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",NaN,tt15654502,hi,The Ladykiller,When a small-town playboy falls in love for a ...,1.700000,...,The Ladykiller,"['mysterious woman', 'erotic thriller', 'suspe...","['Bhumi Pednekar', 'Arjun Kapoor', 'Priyanka B...",NaN,456,False,/eyDx5sUQscDqDC3PdRbsBNH3BxB.jpg,False,0.0,0.0
30336,947650,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",NaN,tt14506102,cs,"Tonko, Slávka a kúzelné svetlo",Tony has been glowing since the day he was bor...,2.102000,...,"Tony, Shelly and the Magic Light",['stop motion'],"['Michael Polák', 'Antonie Baresová', 'Ivana U...",NaN,191018,False,/rFdlSLjaf6qrGChWr9gje6eTcbf.jpg,False,0.0,0.0


In [6]:
# Read the CSV file 'gross_earnings_domestic_2013-2023.csv' into a dataframe and rename the 'daily earnings' column
MOJO_df = pd.read_csv('Additional_Data\gross_earnings_domestic_2013-2023.csv').rename(columns={'daily earnings': 'gross_earnings_domestic'})

In [7]:
# Create sets of movie titles from 'MOJO_df' and 'TMDB_df'
MOJO_set = set(MOJO_df["movie"].to_list())
TMDB_set = set(TMDB_df["title"].to_list())

# Find the intersection of the two sets to determine the number of movies that exist in both dataframes
common_movies = MOJO_set.intersection(TMDB_set)

# Print the total number of common movies
print(f'Total movies in both dataframes: {len(common_movies)}')

Total movies in both dataframes: 3364


In [8]:
# Use an inner join to include only rows with matching movie titles in both dataframes
merged_df = pd.merge(MOJO_df, TMDB_df, left_on='movie', right_on='title', how='inner')

# Remove duplicate rows based on the 'imdb_id' column, keeping the first occurrence
merged_df.drop_duplicates(subset=['imdb_id'], keep='first', inplace=True)


In [9]:
#Removing the missing data or data that will not be useful in the model
merged_df.drop(columns=['belongs_to_collection','poster_path','status','tagline','crew','adult','backdrop_path','video'], inplace=True)


In [10]:
#Filtering only rows that have revenues
final_df=merged_df[merged_df['revenue']!=0] 

In [14]:
final_df[['production_countries','spoken_languages','keywords','production_companies','genres','cast']].head()

,production_countries,spoken_languages,keywords,production_companies,genres,cast
0,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 2187, 'name': 'riot'}, {'id': 6091, 'n...","[{'name': 'Screen Yorkshire', 'id': 2690}, {'n...","[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...","[{'cast_id': 6, 'character': 'Gary Hook', 'cre..."
1,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1930, 'name': 'kidnapping'}, {'id': 23...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'id': 53, 'name': 'Thriller'}, {'id': 878, '...","[{'cast_id': 2, 'character': 'Michelle', 'cred..."
3,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","['kidnapping', 'revenge']","[{'id': 5056, 'logo_path': '/583ITs4w9sK31FnJw...","[{'id': 53, 'name': 'Thriller'}]","['Luke Evans', 'Kelly Reilly', 'Noel Clarke', ..."
4,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","['cia', 'based on novel or book', 'afghanistan...","[{'id': 79529, 'logo_path': '/gVN3k8emmKy4iV4K...","[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...","['Chris Hemsworth', 'Michael Shannon', 'Michae..."
5,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","['slavery', 'plantation', 'based on memoir or ...","[{'id': 10104, 'logo_path': '/wRn5HnYMGeJHmItR...","[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...","['Chiwetel Ejiofor', 'Michael Fassbender', ""Lu..."


In [15]:
import ast
import json

# Define a function to convert a date string to a datetime object
def convert_date_string_to_datetime(date_string):
    try:
        return pd.to_datetime(date_string, errors='coerce')
    except:
        return None

# Define a function to extract country names from a string containing a list of dictionaries
def extract_countries(production_countries_str):
    try:
        # Convert the string representation of the list of dictionaries into an actual list of dictionaries
        production_countries_list = ast.literal_eval(production_countries_str)

        # Extract the country names
        return [country['name'] for country in production_countries_list]
    except:
        return []

# Define a function to extract language names from a string containing a list of dictionaries
def extract_languages(spoken_languages_str):
    try:
        # Convert the string representation of the list of dictionaries into an actual list of dictionaries
        spoken_languages_list = ast.literal_eval(spoken_languages_str)

        # Extract the language names
        return [language.get('name', language.get('english_name', 'Unknown')) for language in spoken_languages_list]
    except:
        return []

# Define a function to extract keywords from a string containing a list of dictionaries or strings
def extract_keywords(keywords_str):
    try:
        # Attempt to interpret the string as a list
        keywords_list = ast.literal_eval(keywords_str)

        # Check if the first element of the list is a dictionary
        if isinstance(keywords_list, list) and keywords_list and isinstance(keywords_list[0], dict):
            # Extract the keyword names from dictionaries
            return [keyword['name'] for keyword in keywords_list]
        elif isinstance(keywords_list, list):
            # The list already contains keyword names as strings
            return keywords_list
        else:
            return []
    except:
        return []

# Define a function to extract production company names and create a comma-separated string
def extract_production_company_names(production_companies):
    # Check if the production companies data is a string and convert to a list if necessary
    if isinstance(production_companies, str):
        try:
            production_companies = json.loads(production_companies.replace("'", "\""))
        except json.decoder.JSONDecodeError:
            return None
    # Extract the names of the production companies
    if isinstance(production_companies, list):
        company_names = [company['name'] for company in production_companies if 'name' in company]
        return ', '.join(company_names)  # Use ', ' to separate with a comma and space
    return None

# Apply the 'extract_production_company_names' function to the 'production_companies' column in 'TMDB_df'
TMDB_df['production_companies'] = TMDB_df['production_companies'].apply(extract_production_company_names)

# Define a function to extract genres from a string and return them as a list
def get_genres(x):
    if pd.isna(x):
        return x
    elif '[' in x:
        genres_list = []
        x = x.replace("'", '"')
        for genre_dict in json.loads(x):
            genres_list.append(genre_dict['name'])
        return genres_list
    else:
        return [x]

# Define a function to extract character names from a string containing a list of dictionaries
def extract_character_names(cast_str):
    try:
        # Convert the string representation of the list of dictionaries into an actual list of dictionaries
        cast_list = ast.literal_eval(cast_str)

        # Extract the character names using list comprehension
        return [cast_member['character'] for cast_member in cast_list]
    except:
        return []

# Define a function to extract spoken languages from a string containing a list of dictionaries
def spoken_languages(data):
    if pd.isna(data):
        return None  # Return None for NaN values
    # Replace single-quotes with double-quotes and parse as JSON
    data = data.replace("'", '"')
    language_list = json.loads(data)
    result = [f"[{item['iso_639_1']}]" for item in language_list]
    return result


In [16]:
# Apply the 'get_genres' function to the 'genres' column to extract and format genre information
final_df['genres'] = final_df['genres'].apply(get_genres)

# Apply the 'extract_languages' function to the 'spoken_languages' column to extract language information
final_df['spoken_languages'] = final_df['spoken_languages'].apply(extract_languages)

# Apply the 'extract_countries' function to the 'production_countries' column to extract country information
final_df['production_countries'] = final_df['production_countries'].apply(extract_countries)

# Convert the 'release_date' column to datetime format, handling any errors by coercing invalid values to NaN
final_df['release_date'] = pd.to_datetime(final_df['release_date'], errors='coerce')

# Apply the 'extract_keywords' function to the 'keywords' column to extract keyword information
final_df['keywords'] = final_df['keywords'].apply(extract_keywords)

# Apply the 'extract_production_company_names' function to the 'production_companies' column
# to extract and format production company information
final_df['production_companies'] = final_df['production_companies'].apply(extract_production_company_names)

# Apply the 'extract_character_names' function to the 'cast' column to extract character names
final_df['cast'] = final_df['cast'].apply(extract_character_names)


C:\Users\anson\AppData\Local\Temp\ipykernel_15656\2480591178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['genres'] = final_df['genres'].apply(get_genres)
C:\Users\anson\AppData\Local\Temp\ipykernel_15656\2480591178.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['spoken_languages'] = final_df['spoken_languages'].apply(extract_languages)
C:\Users\anson\AppData\Local\Temp\ipykernel_15656\2480591178.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [17]:
final_df[['production_countries','spoken_languages','keywords','production_companies','genres','cast']].head()

,production_countries,spoken_languages,keywords,production_companies,genres,cast
0,[United Kingdom],[English],"[riot, war, british, survival, soldier, ira, e...","Screen Yorkshire, British Film Institute (BFI)...","[Thriller, Action, Drama, War]","[Gary Hook, MRF NCO Lewis, Captain Sandy Brown..."
1,[United States of America],[English],"[kidnapping, bunker, paranoia, basement, survi...","Paramount Pictures, Bad Robot, Spectrum Effects","[Thriller, Science Fiction, Drama]","[Michelle, Howard Stambler, Emmett DeWitt, Ben..."
3,[United Kingdom],[English],"[kidnapping, revenge]","Head Gear Films, Unstoppable Entertainment",[Thriller],[]
4,[United States of America],[English],"[cia, based on novel or book, afghanistan, bas...","Torridon Films, Alcon Entertainment, Black Lab...","[War, Drama, Action, History]",[]
5,"[United States of America, United Kingdom]",[English],"[slavery, plantation, based on memoir or autob...","New Regency Pictures, Plan B Entertainment, Ri...","[Drama, History]",[]


In [ ]:
# Read the 'Tomato_Score.csv' file into a dataframe named 'tomato'
tomato = pd.read_csv('Outputs\Tomato_Score.csv')

# Merge 'tomato' and 'final_df' dataframes based on the 'Movie' column from 'tomato' and the 'title' column from 'final_df'
# Use an inner join to include only rows with matching movie titles in both dataframes
finalwithtomato_df = pd.merge(tomato, final_df, left_on='Movie', right_on='title', how='inner')


In [ ]:
finalwithtomato_df.columns

In [ ]:
finalwithtomato_df

In [ ]:
# Read the 'Youtube_Movie_Data.csv' file into a dataframe named 'youtube'
youtube = pd.read_csv('Outputs\Youtube_Movie_Data.csv')

In [ ]:
# Merge 'youtube' and 'finalwithtomato_df' dataframes based on the 'MovieTitle' column from 'youtube' and the 'title' column from 'finalwithtomato_df'
# Use an inner join to include only rows with matching movie titles in both dataframes
final_df = pd.merge(youtube, finalwithtomato_df, left_on='MovieTitle', right_on='title', how='inner')


In [ ]:
# Drop the 'YouTube Video ID' and 'YouTube Video Title' columns from the 'final_df' dataframe
final_df.drop(columns=['YouTube Video ID', 'YouTube Video Title'], inplace=True)


In [ ]:
final_df.drop_duplicates(subset=['imdb_id'], keep='first', inplace=True)


In [ ]:
final_df

In [ ]:
##